In [1]:
import pandas as pd
import numpy as np
import glob
import ntpath
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from visits import Visit, Cohort, MissingDataError
import random

In [2]:
cohort_1_details = pd.read_csv("cohort_1_details_v5.csv", sep=',')
cohort_2_details = pd.read_csv("cohort_2_details_v5.csv", sep=',')
cohort_3_details = pd.read_csv("cohort_3_details_v5.csv", sep=',')
cohort_4_details = pd.read_csv("cohort_4_details_v5.csv", sep=',')

In [3]:
cohort_1_duration = pd.DataFrame(cohort_1_details['extubation_duration'].quantile(q=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]))


In [4]:
cohort_2_details['available_data_duration'] = cohort_2_details['available_data_duration']/ timedelta(minutes=1)


In [5]:
cohort_1_duration

,extubation_duration
0.0,123.0
0.1,180.0
0.2,229.0
0.3,285.0
0.4,360.0
0.5,420.0
0.6,600.0
0.7,840.0
0.8,1128.0
0.9,1272.0


In [6]:
max(cohort_2_details["available_data_duration"])

703616.4

In [7]:
cohort_2_details["time_category"] = 0

In [8]:
for i, a in zip(cohort_2_details.index, cohort_2_details["available_data_duration"]):
    if a < 180:
        cohort_2_details["time_category"].loc[i] = 1
    if 180 <= a < 229:
        cohort_2_details["time_category"].loc[i] = 2
    if 229 <= a < 285:
        cohort_2_details["time_category"].loc[i] = 3
    if 285 <= a < 360:
        cohort_2_details["time_category"].loc[i] = 4
    if 360 <= a < 420:
        cohort_2_details["time_category"].loc[i] = 5
    if 420 <= a < 600:
        cohort_2_details["time_category"].loc[i] = 6
    if 600 <= a < 840:
        cohort_2_details["time_category"].loc[i] = 7
    if 840 <= a < 1128:
        cohort_2_details["time_category"].loc[i] = 8
    if 1128 <= a < 1272:
        cohort_2_details["time_category"].loc[i] = 9
    if 1272 <= a < 703617:
        cohort_2_details["time_category"].loc[i] = 10

/Users/lucindakhalil/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
pd.cut(cohort_2_details['available_data_duration'],[0, 180, 229, 285, 360, 420, 600, 840, 1128, 1272, 703617]).value_counts()


(1272, 703617]    1531
(840, 1128]        164
(1128, 1272]       117
(600, 840]          71
(420, 600]          54
(285, 360]          22
(229, 285]          16
(180, 229]          12
(360, 420]          12
(0, 180]            11
Name: available_data_duration, dtype: int64

In [14]:
len(cohort_2_details)
# 201 in each strata

2010

In [15]:
cohort_2_details["extubation_deid_date"] = pd.to_datetime(cohort_2_details["extubation_deid_date"])

In [16]:
cohort_2_details["pseudo_reintubation"] = 0

In [18]:
for i, s, c in zip(cohort_2_details.index, cohort_2_details["extubation_deid_date"], cohort_2_details["time_category"]):
    if c in [1,2,3,4,5,6,7]:
        cohort_2_details["pseudo_reintubation"].loc[i] = s + timedelta(minutes=random.randint(120,180))
    if c == 8:
        cohort_2_details["pseudo_reintubation"].loc[i] = s + timedelta(minutes=random.choices([random.randint(120,180),random.randint(180,229)],[0.01829268,0.981707])[0])
    if c == 9:
        cohort_2_details["pseudo_reintubation"].loc[i] = s + timedelta(minutes=random.choices([random.randint(180,229),random.randint(229,285)],[0.34188,0.6581197])[0])
    if c == 10:
        cohort_2_details["pseudo_reintubation"].loc[i] = s + timedelta(minutes=random.choices([random.randint(229,285),random.randint(285,360),random.randint(360,420),random.randint(420,600), random.randint(600,840),random.randint(840,1128),random.randint(1128,1272),random.randint(1272,703617)],[0.0809928,0.1312867,0.1312867,0.1312867,0.1312867,0.1312867,0.1312867,0.1312867])[0])

In [19]:
cohort_2_details

,Unnamed: 0,Project ID,birth_deid_date,death_deid_date,sex,icu_visit,icu_start_deid_dttm,icu_end_deid_dttm,failed_extubation_flag,failed_extubation_deid_date,...,extubation_score,re_intubation_deid_date,icu_ward,total_visits,final_datapoint,available_data_duration,age (days),age_category,time_category,pseudo_reintubation
0,4183,FJ2998,2017-09-24,NaN,Male,1,2017-09-28 16:15:00,2017-10-13 23:35:00,NaN,NaN,...,4.0,NaN,NICU,1,2017-10-14 00:04:48,21784.800000,4.875000,1,10,2017-09-29 09:19:00
1,1807,ZA1349,2016-03-02,NaN,Female,1,2016-03-02 22:15:00,2016-03-08 19:14:00,NaN,NaN,...,5.0,NaN,FLAMI,1,2016-03-08 11:40:46,2860.766667,4.500000,1,10,2016-03-06 20:19:00
2,4188,HM3001,2017-09-20,NaN,Male,1,2017-09-23 16:00:00,2017-10-13 16:30:00,NaN,NaN,...,5.0,NaN,NICU,2,2017-10-13 15:58:36,24538.600000,6.625000,1,10,2017-09-26 21:53:00
3,3996,QV2851,2017-07-28,NaN,Male,2,2017-07-30 08:00:00,2017-08-05 15:41:00,NaN,NaN,...,5.0,NaN,NICU,2,2017-08-07 10:46:49,3211.816667,8.218750,1,10,2017-08-06 00:29:00
4,5489,QH3969,2018-10-12,NaN,Male,1,2018-10-13 15:00:00,2018-10-24 16:40:00,NaN,NaN,...,5.0,NaN,NICU,1,2018-10-24 19:50:03,7355.050000,7.718750,1,10,2018-10-19 22:14:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005,299,JD0238,2000-01-20,NaN,Male,1,2018-06-10 14:11:00,2018-06-11 13:15:00,NaN,NaN,...,5.0,NaN,PICU,1,2018-06-11 12:01:04,1081.066667,6716.750000,10,8,2018-06-10 21:45:00
2006,182,BM0148,2004-06-09,NaN,Male,1,2016-10-30 00:45:00,2016-11-03 18:30:00,NaN,NaN,...,5.0,NaN,PICU,1,2016-11-02 15:11:39,1691.650000,4528.458333,10,10,2016-11-02 04:11:00
2007,355,SQ0287,2003-01-11,NaN,Female,1,2018-07-20 16:53:00,2018-07-22 11:25:00,NaN,NaN,...,5.0,NaN,PICU,1,2018-07-22 02:07:39,1027.650000,5670.375000,10,8,2018-07-21 12:03:00
2008,183,CH0149,2003-05-07,NaN,Female,1,2017-07-05 18:10:00,2017-07-09 13:35:00,NaN,NaN,...,5.0,NaN,PICU,1,2017-07-10 20:17:32,5837.533333,5174.791667,10,10,2017-07-07 11:20:00


In [20]:
# overwriting csv frin data_preparation_v3 file
cohort_2_details.to_csv('cohort_2_details_v5.csv')